# Homework13

Audio and Neural Networks

## Goals

- Get more familiar with neural network setup, design, data preparation and training
- Practice setting up the objects and parameters for the training loop
- Experiment with a pre-trained image neural networks for an audio classification tasks

### Setup

Run the following 2 cells to import all necessary libraries and helpers for this homework

In [ ]:
!wget -q https://github.com/PSAM-5020-2025F-A/5020-utils/raw/main/src/audio_utils.py
!wget -q https://github.com/PSAM-5020-2025F-A/5020-utils/raw/main/src/data_utils.py
!wget -q https://github.com/PSAM-5020-2025F-A/5020-utils/raw/main/src/image_utils.py
!wget -q https://github.com/PSAM-5020-2025F-A/5020-utils/raw/main/src/nn_utils.py

!wget -qO- https://github.com/PSAM-5020-2025F-A/5020-utils/releases/latest/download/birds.tar.gz | tar xz

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from IPython.display import Audio
from os import listdir, path

from torch import nn, Tensor, float32 as t_float32, uint8 as t_uint8
from torch.optim import SGD

from torchvision.models import resnet34, ResNet34_Weights
from torchvision.transforms import v2

from audio_utils import get_samples_and_rate, stft
from data_utils import classification_error, display_confusion_matrix
from image_utils import make_image
from nn_utils import get_labels, get_num_params

## Intro

We're going to see how to train a CNN to classify bird sounds.

Yep. These networks are so good at analyzing images that they have also been used to classify audio and text.

This usually requires some kind of pre-transformation in order to turn non-visual data into images.

This is where we have to get creative, since images are these two-dimensional objects that don't really depend on time, while audio and text are mostly one-dimensional entities with a beginning and an end (we read left-to-right, top-to-bottom, and audio is air pressure change over time).

### 2D Audio

There is a well-defined way to represent audio as a two-dimensional measure using the [Short-Time Fourier Transform](https://en.wikipedia.org/wiki/Short-time_Fourier_transform). We briefly saw this in the very end of our notebook for [Week 04](https://github.com/PSAM-5020-2025F-A/WK04).

The Short-Time Fourier Transform can be used to calculate which frequencies are present in subsections (windows) of our audio files. If our audio is $5$ seconds long, we can split it into $200$ subsections of $25$ milliseconds each and compute what frequencies are present in each of these windows.

This gives us a two-dimensional representation of our audio, where one direction represents time, the other represents frequency, and the numbers represent how much of a given frequency is present at that time.

We have a `stft()` function in our `audio_utils` to simplify this process. It takes a list of audio samples, the sampling rate and the length of the sub-sectioning window to use for the frequency analysis, and returns three lists:

- `freqs`: the list of frequency values reported by the analysis. The first element gives us the lowest frequency detected and the last element the highest.
- `times`: the time stamps for the windows used in the analysis. If our original file has $80\text{,}000$ samples and we divide this into windows of $400$ samples, this `times` list will have $200$ values.
- `ffts`: this is a list-of-lists, where each row corresponds to a frequency analyzed and each column represents a moment in time. This list should have as many rows as `len(freqs)` and as many columns as `len(times)`.

In [ ]:
# Audio file location
filepath = "./data/audio/birds/train/duck_0001.wav"

# Widget to play the audio
display(Audio(filepath))

# Read audio file and get its samples and sampling rate
samples,rate = get_samples_and_rate(filepath)

# Perform the short-time frequency transformation
ffts, freqs, times = stft(samples, rate, window_len=400)

# Plot results. Times in the x-axis, frequencies in the y-axis,
# and the values in ffts represent the strength of a particular frequency at a particular time
plt.pcolormesh(times, freqs, ffts)
plt.show()

## Bird Sound Dataset: Labels

We have a dataset with $3\text{,}150$ different audio files of bird sounds. That's $150$ files for each of the $21$ different types of birds in the dataset. The dataset has already been split into `train` and `test` directories, using a $66\%$ $/$ $33\%$ split, so twice as many files in the `train` dataset.

All we have in the `./data/audio/birds/train` and `./data/audio/birds/test` directories are the audio files. Luckily their names are easy to parse and split into the correct bird label.

Write a function that extracts the `birdname` from file path in the form `./data/audio/birds/train/birdname_nnnn.wav`.

In [ ]:
# TODO: write a filepath_to_label() function that turns file paths into bird names
# HINT: the split() function can be used twice to separate the file path by '/' and then '_'


def filepath_to_label(filepath: str) -> str:
    # Split path to get filename
    filename = filepath.split('/')[-1]
    
    # Split filename to get bird name
    label = filename.split('_')[0]
    
    return label


## Bird Sound Dataset: Images

This is a bit more complex.

We do have the `stft()` function in `audio_utils`, but it returns lists formatted in a way that is easy to visualize the results using `matplotlib`.

What we want is a flat list of grayscale pixel values that we can then put in a tensor and pre-process for a CNN.

What we have to do is flatten the resulting list-of-lists into a single list of values and then scale the values so they are all between $0$ and $255$.

This is basically `MinMaxScaling()`, but we're doing it by hand here so that it runs a little bit faster once we use it on our $3\text{,}150$ files.

In [ ]:
# Function for turning an audio file into a list of grayscale "pixels"

def wav_to_pxs(filepath, window_len=400):
  samples,rate = get_samples_and_rate(filepath)
  ffts = np.array(stft(samples, rate, window_len)[0])
  ffts_min, ffts_max = ffts.min(), ffts.max()
  pxs = 255 * (ffts - ffts_min) / (ffts_max - ffts_min)
  return pxs.reshape(-1).tolist()

### Image from audio

We can check that the above function works by testing it on a single file.

We just have to call `wav_to_pxs()` with a file path and a length for the sub-sectioning windowing.

The result is a list of grayscale pixel values.

One thing that helps here is that all files in our dataset already have the same length and sampling rate. This means that the resulting list of pixels will have the same length as well, as long as we use a consistent `window_len` value for all of them.

We can recover the "`height`" of our images by dividing the `window_len` by $2$. This has to do with how the `stft()` (and its internal `fft()` function) works and how audio with more samples give more granular frequency resolution; more elements in the `freqs` list, more rows in our `ffts` variable.

Larger `window_len` values will also mean fewer time steps analyzed (fewer values in the `times` list). So, our `stft()` function always returns the same number of pixels, we just have to decide how we distribute those pixels in the $x$ and $y$ directions using the `window_len` parameter.

For our particular dataset, a `window_len` of $400$ gives us square images, but experiment with the `window_len` parameter to see how it affects the resulting image size.

In [ ]:
# Audio file location
filepath = "./data/audio/birds/train/duck_0001.wav"

# TODO: experiment with this value and see how it changes the image
window_len = 200

# Get pixels from the stft() function
pxs = wav_to_pxs(filepath, window_len)

# This is how we recover the number of rows and columns in our "image"
ih = window_len // 2
iw = len(pxs) // ih

# Check image
display(make_image(pxs, iw))

### Interpretation

<span style="color:hotpink;">
So... What happens ?<br>
What are the effects of changing the <code>window_len</code> parameter ?<br>
Do we get more or less information about our audio?
</span>

<span style="color:hotpink;">

Changing window_len does not add or remove information from the audio. It only changes how the information is represented.

Smaller window_len: more time detail, less frequency detail

Larger window_len: more frequency detail, less time detail

So the audio content stays the same, but the image looks different, which can affect how well the neural network learns patterns.

</span>

## Get the Data

Now's the time to go through all of the audio files in our dataset and extract their "pixels" and labels.

This is similar to how we processes the files for the security camera and LFW datasets.

We'll iterate over all of the files, open them, and put their labels in a list and their "pixels" in another list.

Use square images. CNNs run more efficiently on square images due to the way the convolution kernels are defined.

Also, we're processing $2\text{,}100$ audio files here... this cell might take up to $5$ minutes to run.

In [ ]:
# This is where the training files are
TRAIN_DIR = "./data/audio/birds/train"

# This is a list of all of the training file names
train_fnames = sorted([f for f in listdir(TRAIN_DIR) if f.endswith("wav")])

# To store the pixels and labels of each file
train_pixels = []
train_labels = []




In [ ]:
# TODO: iterate through the files, open them and extract pixels and labels

window_len = 400  # square images

for fname in train_fnames:
    filepath = path.join(TRAIN_DIR, fname)
    
    # Extract pixels
    pxs = wav_to_pxs(filepath, window_len)
    train_pixels.append(pxs)
    
    # Extract label
    label = filepath_to_label(filepath)
    train_labels.append(label)

### Repeat for the `test` data

Again... we're processing $1\text{,}050$ audio files here... this cell might take up to $3$ minutes to run.

In [ ]:
# This is where the test files are
TEST_DIR = "./data/audio/birds/test"

# This is a list of all of the test file names
test_fnames = sorted([f for f in listdir(TEST_DIR) if f.endswith("wav")])

# To store the pixels and labels of each file
test_pixels = []
test_labels = []

# TODO: iterate through the files, open them and extract pixels and labels

window_len = 400  # must match training data

for fname in test_fnames:
    filepath = path.join(TEST_DIR, fname)
    
    # Extract pixels
    pxs = wav_to_pxs(filepath, window_len)
    test_pixels.append(pxs)
    
    # Extract label
    label = filepath_to_label(filepath)
    test_labels.append(label)


## Tensor Everything

Now we put everything into `Tensor` objects for our CNN.

<!-- <img src="./imgs/cnn_layers.jpg" height="200px" /> -->
<!-- <img src="./imgs/cnn_fc.jpg" height="200px" /> -->

<img src="https://i.postimg.cc/rpdq7DSd/cnn-layers.jpg" height="200px" />
<img src="https://i.postimg.cc/XYYjcHqk/cnn-fc.jpg" height="200px" />

### Label Tensors

We need to encode out text labels into numbers before we put them into `Tensors`.

We can certainly use an `OrdinalEncoder` from `sklearn`, but a quicker way is to just get a list of all of the unique values in the `train_labels` or `test_labels` lists. Then, we can encode a label text `label_str` by just using the lists `index()` function, like this:

`label_int = unique_labels.index(label_str)`

### Image Tensors

Since CNNs work with $2D$ image information, we have to reshape our pixels back into two-dimensional lists-of-lists. CNNs also expect our images to be represented as single layers, so different images of single-channel pixels and not a single list of multi-channel pixels.

Our images are black and white here, so this isn't a big deal, but we still have to use the [`movedim()`](https://pytorch.org/docs/stable/generated/torch.movedim.html#torch.movedim) or [`permute()`](https://pytorch.org/docs/stable/generated/torch.permute.html) functions to re-arrange our `Tensor` dimensions and make it match what is expected by the CNN.

If this doesn't sound familiar, take a look at the "**New Network, New Shape**" section of our [WK13](https://github.com/PSAM-5020-2025F-A/WK13) notebook for some guidance.

Remember that the `height` for our images here is `window_len` divided by $2$, and their `width` is just the number of pixels divided by their `height`. If using a `window_len` of $400$, the images should be squares of $200$ $\times$ $200$ pixels.

In [ ]:
# TODO create a label encoder
unique_labels = sorted(list(set(train_labels)))

# TODO encode labels and put them into a Tensor of type long() (whole numbers)
train_y = Tensor([unique_labels.index(lbl) for lbl in train_labels]).long()
test_y  = Tensor([unique_labels.index(lbl) for lbl in test_labels]).long()

# TODO encode pixels and put them into a Tensor with shape N x C x H x W
window_len = 400
H = window_len // 2
W = len(train_pixels[0]) // H

# Convert train pixels
train_X = Tensor(train_pixels).reshape(len(train_pixels), H, W)
train_X = train_X.unsqueeze(1)  # add channel dimension (C = 1)

# Convert test pixels
test_X = Tensor(test_pixels).reshape(len(test_pixels), H, W)
test_X = test_X.unsqueeze(1)  # add channel dimension (C = 1)

# TODO check shapes
print("Train X:", train_X.shape)
print("Train y:", train_y.shape)
print("Test X:", test_X.shape)
print("Test y:", test_y.shape)





In [ ]:

train_pixels_t = train_X
test_pixels_t  = test_X

train_labels_t = train_y
test_labels_t  = test_y

print("train_pixels_t:", train_pixels_t.shape)
print("train_labels_t:", train_labels_t.shape)
print("test_pixels_t:", test_pixels_t.shape)
print("test_labels_t:", test_labels_t.shape)


## Prepare for ResNet

Our data is ready for generic, untrained, plain CNNs, but since we're going to use a pre-trained [Residual Network](https://arxiv.org/abs/1512.03385), we still have a couple of steps left before we're ready to train our model.

<!-- <img src="./imgs/resnet34_01.jpg" height="200px" /> -->
<img src="https://i.postimg.cc/hP20Rn9D/resnet34-01.jpg" height="200px" />

We're going to use the `ReNet34` [pre-trained model](https://pytorch.org/hub/pytorch_vision_resnet/) available in the `PyTorch` library. This is not the largest `ResNet` model, but will fit nicely into small GPUs.

The `ResNet` models in `PyTorch` were all trained on the [ImageNet](https://image-net.org/download.php) dataset. This dataset has $1\text{,}281\text{,}167$ training images and classifies objects into $1\text{,}000$ classes.

As such, we have to further process our data in order to represent it using the exact same format that was used in the initial training process.

From the [documentation](https://pytorch.org/hub/pytorch_vision_resnet/):

_All pre-trained models expect input images normalized in the same way, i.e. mini-batches of 3-channel RGB images of shape (3 x H x W), where H and W are expected to be at least 224. The images have to be loaded in to a range of [0, 1] and then normalized using mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225]._

We can use the following  `PyTorch` transformation functions to achieve this.

- `ToDtype(t_uint8)`: makes sure our pixels are represented as whole numbers between $0$ and $255$.
- `Resize(224)`: makes the images at least $224$ pixels on each side.
- `Grayscale(3)`: turns the image into grayscale, but keeps $3$ channels.
- `ToDtype(t_float32, scale=True)`: transforms $[0, 255]$ values (`int`) into $[0, 1]$ (`float` ).
- `Normalize()`: same as `StandardScaler`, but using pre-fitted values that were derived from the ImageNet dataset.

In [ ]:
res_transforms = v2.Compose([
  v2.ToDtype(t_uint8),
  v2.Resize(224),
  v2.Grayscale(3),
  v2.ToDtype(t_float32, scale=True),
  v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

### Test Transformation

We can run the transformation on a single image and use the `ToPILImage()` function to inspect the transformed image. The `ToPILImage()` function undoes the channel transposition and transforms a `PyTorch` image ($3$ separate single-channel images) back into a `PIL` image (single image with $3$ channels).

The transformed images will look noisy and should have splotchy colors. This is due to normalizing each of the three channels using different mean and standard deviation values.

It doesn't look good for us, but it's what the `ResNet` expects.

In [ ]:
img_t = res_transforms(train_pixels_t[:1])
display(v2.ToPILImage()(img_t[0]))

### Transform and put on GPU

The following cell runs the transformation on the pixel `Tensor` objects and puts them, along with the label `Tensor` objects, on the GPU.

This assumes that the labels are in `Tensor` objects called `train_labels_t` and `test_labels_t`, and pixels in `train_pixels_t` and `test_pixels_t`.

Adjust if necessary.

In [ ]:
import torch

device = torch.device("cpu")
print("Using device:", device)

x_train_res = res_transforms(train_pixels_t).to(device)
x_test_res  = res_transforms(test_pixels_t).to(device)

y_train = train_labels_t.to(device)
y_test  = test_labels_t.to(device)

print("Training dataset shape:", x_train_res.shape)
print("Image shape:", x_train_res[0].shape)


## Training Setup

These are the `PyTorch` objects that we need in order to train our model:

- `model`: our network, this will be an instance of a `ResNet34` model
- `optim`: optimizer to adjust our model parameters
- `loss_fn`: for classification models we can use the `CrossEntropyLoss` function to compute the error of our predictions during training

### Modify `ResNet34`

The last layer of the default `ResNet34` model has $1\text{,}000$ outputs, one for each of the classes in the ImageNet dataset, but we're not using that dataset.

Since we're classifying our audios into one of $21$ classes, we have to swap the last layer of the `ResNet` model for one with $21$ outputs. The [WK13](https://github.com/PSAM-5020-2025F-A/WK13) notebook has an example of how to do this.

In [ ]:
# TODO instantiate a resnet34 network
model = resnet34(weights=ResNet34_Weights.DEFAULT)

# TODO change the network's last layer to match the number of classes
num_classes = len(unique_labels)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# TODO put model on device (CPU here)
import torch
device = torch.device("cpu")
model = model.to(device)

# TODO check number of parameters that will be trained
print("Trainable parameters:", get_num_params(model))

# TODO instantiate an optimizer
optim = SGD(model.parameters(), lr=0.001, momentum=0.9)

# TODO instantiate a loss function
loss_fn = nn.CrossEntropyLoss()


### Test Network Shapes

Run the following cell to make sure the network is connected properly.

We're not able to give all of the images to our network at once, so we're using the `batch_step` variable to grab subsets of $15$ images at a time.

Our input should have a shape of $15$ x $3$ x $200$ x $200$, representing the number of images, number of channels, height and width of the images in our batch. Likewise, the output should be of shape $15$ x $21$, representing the $21$ class activation values for each of the $15$ images in our batch.

In [ ]:
batch_step = int(len(x_train_res) // 15)

out = model(x_train_res[::batch_step])

print("Input shape:", x_train_res.shape)
print("Output shape:", out.shape)
print("Parameters:", get_num_params(model))

## Training

Create a training loop like we saw in [class](https://github.com/PSAM-5020-2025F-A/WK13).

Since we can't put all of our images into the model at once, we'll train our model using batches of $15$ images. The `batch_step` variable should already be set by running the cell above, and we can use it like this to get batches inside our main training loop:

```python
for batch_start in range(batch_step):
  batch_input = all_inputs[batch_start::batch_step]
  batch_output = all_outputs[batch_start::batch_step]
```

For each batch in our loop, we should:
- Clear the optimizer annotations
- Predict classes by feeding inputs into the `model`
- Measure `loss` (this is just `loss_fn(predicted, actual)`)
- Get the optimizer to back-propagate and annotate the neurons
- Update parameters

In order to check if the model is overfitting, we can sporadically run evaluations within the training loop in order to see if the model performs similarly with `train` and `test` data.

We can use the `get_labels(model, inputs)` function inside the `nn_utils` file to run our `model` on all of the data in a given batch and return the predicted labels for all of the samples. This takes care of evaluating which neuron in our last layer is the most activated.

It shouldn't take many epochs to get the training loss close to zero, but even so, running through all $2\text{,}100$ training images might take about $20$ seconds per epoch.

In [ ]:
# TODO: iterate over epochs
# TODO: iterate over batches
# TODO: clear optimizer annotations
# TODO: get predictions
# TODO: compute loss
# TODO: annotate neurons with loss values
# TODO: adjust model parameters
# TODO: every once in a while, print loss and classification error values


import torch

epochs = 3        # fewer epochs
batch_step = 20   # smaller batches

# use small subsets for evaluation
eval_train = x_train_res[:200]
eval_train_y = y_train[:200]

eval_test = x_test_res[:200]
eval_test_y = y_test[:200]

for epoch in range(epochs):
    
    model.train()
    
    for batch_start in range(batch_step):
        batch_x = x_train_res[batch_start::batch_step]
        batch_y = y_train[batch_start::batch_step]
        
        optim.zero_grad()
        preds = model(batch_x)
        loss = loss_fn(preds, batch_y)
        loss.backward()
        optim.step()
    

    model.eval()
    with torch.no_grad():
        train_preds = get_labels(model, eval_train)
        test_preds  = get_labels(model, eval_test)
    
    train_err = classification_error(train_preds, eval_train_y)
    test_err  = classification_error(test_preds, eval_test_y)
    
    print(
        f"Epoch {epoch} | "
        f"Loss: {loss.item():.4f} | "
        f"Train Error: {train_err:.3f} | "
        f"Test Error: {test_err:.3f}"
    )


## Evaluate

Now that the model has been trained for our specific bird sound classification task, we can finally calculate some accuracy statistics for it.

The `get_labels(model, inputs)` function can be useful here in order to run the model on all of the samples in our datasets and get final predicted label values from the model's final activation layer. These are the numbers we have to use when computing accuracy (maybe precision and recall) and displaying confusion matrices.

In [ ]:
import torch

model.eval()

# Use manageable subsets to avoid kernel crash
eval_train = x_train_res[:300]
eval_train_y = y_train[:300]

eval_test = x_test_res[:300]
eval_test_y = y_test[:300]

with torch.no_grad():
    # Predictions
    train_preds = get_labels(model, eval_train)
    test_preds  = get_labels(model, eval_test)
    
    # Loss
    train_logits = model(eval_train)
    test_logits  = model(eval_test)
    
    train_loss = loss_fn(train_logits, eval_train_y).item()
    test_loss  = loss_fn(test_logits, eval_test_y).item()

# Classification error
train_err = classification_error(train_preds, eval_train_y)
test_err  = classification_error(test_preds, eval_test_y)

# TODO: print final loss and classification error values
print("Final Training Loss:", round(train_loss, 4))
print("Final Training Error:", round(train_err, 3))
print("Final Test Loss:", round(test_loss, 4))
print("Final Test Error:", round(test_err, 3))

# TODO: display confusion matrices for train and test data
display_confusion_matrix(train_preds, eval_train_y, unique_labels)
display_confusion_matrix(test_preds, eval_test_y, unique_labels)
